In [1]:
import numpy as np
import scipy as sp

In [51]:
def SPCA(C, k):
    omega = np.random.randn(C.shape[1], k)
    
    _C = np.zeros(C.shape)
    # Trimming
    for i in range (C.shape[0]):
        if (np.count_nonzero(C[i,:]) > 10):
            _C[i,:] = 0
        else:
            _C[i,:] = C[i,:]
    F = _C.T @ _C - np.diag(np.diag( _C.T @ _C))
    
    ## TODO: power iteration ???
    ## Is log2 ???
    Q, R = np.linalg.qr(np.linalg.matrix_power(F, int(5 * np.log2(C.shape[1]))))
    
    return Q

In [55]:
def SLA(M, k, delta, l):
    #l_samples = int (np.random.choice(M.shape[1], l))
    l_samples = np.random.randint(low = 0, high = M.shape[1], size = l)
    
    print (l_samples.shape)
    
    A_b1 = A_b2 = M[:, list(l_samples)]
    
    Q = SPCA(A_b1, k)
    
    for i in range (A_b2.shape[0]):
        if (np.count_nonzero(A_b2[i,:]) > 2):
            A_b2[i,:] = 0
    for i in range (A_b2.shape[1]):
        if (np.count_nonzero(A_b2[:,i]) > 10 * delta * A_b2.shape[0]):
            A_b2[:,i] = 0
    
    W = A_b2 @ Q
    V = np.zeros((M.shape[1], k))
    V[0:l,:] = (A_b1).T @ W
    
    ## FIXME: ??? what size ???
    I = np.zeros((M.shape[0], k))
    I = A_b1 @ V[l_samples, :]
    
    ### TODO: Need to remove A_b1, A_b2 and Q from RAM
    
    for t in range (l+1, M.shape[1]):
        #t_samples = np.random.choice(M.shape[1], t)
        t_samples = np.random.randint(low = 0, high = M.shape[1], size = t)
        A_t = M[:, t_samples]
        V[t_samples, :] = (A_t).T @ W
        I = I + A_t @ V[t_samples, :]
        
        ## TODO: remove A_t from RAM
    Q, R = np.linalg.qr(V)
    U = (1/delta)(I @ R @ R.T)
    
    
    
    ## FIXME: Need to find out what is here in actual algorithm
    M_k = U @ V.T
    print (M_k.shape)
    
    return M_k
        

In [56]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import requests

url = 'https://cdn.pixabay.com/photo/2013/06/29/21/18/wolf-142173_1280.jpg'

image_raw = Image.open(requests.get(url, stream=True).raw).convert("L")
image = np.array(image_raw)
print (image.shape)

(853, 1280)


In [57]:
l = int(image.shape[1]/10)
delta = int (1/(l * np.log2(image.shape[0])))
M = SLA(image, int (np.linalg.matrix_rank(image) / 5), delta, l)

(128,)


ValueError: shape mismatch: value array of shape (128,128) could not be broadcast to indexing result of shape (128,170)